In [19]:
import pandas as pd
import numpy as np
import requests
import os

### Giving the date of the file as an input, to extract file from mutual fund house official website

In [29]:
#filename = "March 2020.xls"
Date = "March 2020"

### Importing excel file from HDFC MUTUAL FUNDS website

In [30]:
url = "https://files.hdfcfund.com/s3fs-public/2020-04/Monthly Portfolios for March 2020_1.xls"

response = requests.get(url)

filename = "hdfcMarch2020.csv"

with open(filename,"wb") as file1:
    for row in response:
        file1.write(row)
if response.status_code == 200:        
    print("File is imported")

File is imported


url = "https://files.hdfcfund.com/s3fs-public/2020-06/Monthly Portfolios for May 2020_1.xls"

response = requests.get(url)

if response.status_code == 200:        
    print("File is imported")
    
with open(filename,"wb") as file1:
    for row in response:
        file1.write(row)


### Fetching all the sheetnames in imported Excel file

In [31]:
#get all sheet name
xl = pd.ExcelFile(filename)
sheetnames = xl.sheet_names
sheetnames.pop(0)

*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255


*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255


'Hyperlinks'

# Cleansing the excel sheet

In [33]:
count = 0
not_tallied = {}
for fund in sheetnames:
    
    print(count ,fund)
    print("---------")
    
    df = pd.read_excel(filename, sheet_name = fund, skiprows = 4,usecols = [1,2,3,4,5,6,7])
   

    df.columns = ["ISIN","Coupon","instrument","industry/rating","qty","marketval","pct"]
    df["Date"] = Date
    

    df['Coupon'].fillna(0,inplace =True)


#Finding Net Asset value before cleansing the data

    for value in df.ISIN:
        if "Grand Total" in str(value):
            total = df.loc[df['ISIN'] == value]['marketval']
    actual_total = float(total)
    print('Actual ToTal is :',actual_total)
    
    
# Asset names to be changed for data consistency
    #TREPS - Tri-party Repo
    #Net Current Assets
    #Gold - Gold
    #NIFTY NSE PUT OPTION STRIKE PRICE 9600 EXPIR JUN 2020
    #NIFTY NSE PUT OPTION STRIKE PRICE 9900 EXPIR JUN 2020
    #HDFC Corporate Bond Fund - Growth Option - Direct Plan


    #HDFC Sensex Exchange Traded Fund ----ISIN given
    
    #Collateralised Borrowing & Lending Obligation
    #Net Current Assets
    #Safari Industries (India) Ltd.





    df.loc[(df.instrument == 'Net Current Assets'),['Coupon','industry/rating','ISIN','qty']] = [0,'additional','additional',1]
    df.loc[(df.instrument == 'Gold - Gold'),['Coupon','industry/rating','ISIN','qty']] = [0,'additional','additional',1]
    
    df.loc[(df.instrument == 'NIFTY NSE PUT OPTION STRIKE PRICE 9600 EXPIR JUN 2020'),['ISIN','industry/rating']] = ['additional','additional']
    df.loc[(df.instrument == 'NIFTY NSE PUT OPTION STRIKE PRICE 9900 EXPIR JUN 2020'),['ISIN','industry/rating']] = ['additional','additional']
    df.loc[(df.instrument == 'HDFC Corporate Bond Fund - Growth Option - Direct Plan'),['ISIN','industry/rating']] = ['additional','additional']
    
    
    df.loc[(df.ISIN == 'Collateralised Borrowing & Lending Obligation'),['Coupon','industry/rating','instrument','qty']] = ['0','additional','additional',1]
    df.loc[(df.ISIN == 'Net Current Assets'),['Coupon','industry/rating','instrument','qty']] = ['0','additional','additional',1]
                          
    
    for value in df.instrument: 
        if 'Option' in str(value):
            df.loc[(df.instrument == value),['Coupon','industry/rating']] = [0,'additional']
        
        if 'Traded Fund' in str(value):
            df.loc[(df.instrument == value),['Coupon','industry/rating']] = [0,'additional']
            
        if 'Gold Bar' and 'Safari Industries (India) Ltd' in str(value):
            df.loc[(df.instrument == value),['industry/rating']] = ['additional']
            
        if 'Repo' in str(value):
            df.loc[(df.instrument == value),['Coupon','industry/rating','ISIN','qty']] = ['0','additional','additional',1]
        
        if 'Ltd.' or 'Limited' or 'Bank' in str(value):
            isin = list(df.loc[(df.instrument == value)]['ISIN'])
            try:
                if isin[-1] is np.nan:
                    df.loc[(df.instrument == value),['Coupon','industry/rating','ISIN','qty']] = ['0','additional','additional',1]
            except IndexError:
                continue
        else:
            continue
            
            

    #print(df.isnull())
    df.dropna(inplace = True)
    df.reset_index(drop = True,inplace = True)

    #calculated total
    for value in df.marketval:
        df.loc[(df.marketval == "Nil"),['marketval']] = [0]
    calculated_total = df['marketval'].sum()
    print("Calculated Total : ", calculated_total)
    
    
    #taillied is printed if total asset value is equal to the total asset value after cleansing
    #print not tallied file dictionary
    #Storing excel file in a folder HDFC_funds_df
    
    diff = actual_total-calculated_total 
    if abs(diff) < 1:
        print("Tallied")
    else:
        print("There is a differenc of : ",actual_total - calculated_total )
        not_tallied[fund] = diff
        
        
    if not os.path.exists('HDFC_funds_df'):
        os.makedirs('HDFC_funds_df')
   
    df.to_csv('HDFC_funds_df/{}.csv'.format(Date))
    print('CSV Published')
    count += 1
not_tallied

0 HDFCSX
---------
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar 

*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255


*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255


*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255


*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
Actual ToTal is : 120830.58000000002
Calculated Total :  120830.58000000002
Tallied
CSV Published
3 HDFCNYEXTF
---------
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255

...

*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255


*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255


Actual ToTal is : 32960.96
Calculated Total :  32960.96
Tallied
CSV Published
48 HDFCDPEFOF
---------
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncV

*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255


{}

# check not tallied files

In [21]:
exception_sheetnames = []
for fund in exception_sheetnames:
    
    print(count ,fund)
    print("---------")
    
    df = pd.read_excel(filename, sheet_name = fund, skiprows = 4,usecols = [1,2,3,4,5,6,7,8,9])
   

    df.columns = ["ISIN","Coupon","instrument","industry/rating","qty","marketval","pct",'derivative','unhedged']
    df["Date"] = Date
    

    df['Coupon'].fillna(0,inplace =True)
    df['derivative'].fillna(0,inplace =True)
    df['unhedged'].fillna(0,inplace =True)
    


    #Finding Net Asset
    value = df.loc[df['ISIN'] == 'Grand Total']['marketval']
    print('Actual ToTal is :')
    actual_total = float(value)
    print(actual_total)


    #TREPS - Tri-party Repo
    #Net Current Assets
    #Gold - Gold
    #NIFTY NSE PUT OPTION STRIKE PRICE 9600 EXPIR JUN 2020
    #NIFTY NSE PUT OPTION STRIKE PRICE 9900 EXPIR JUN 2020
    #HDFC Corporate Bond Fund - Growth Option - Direct Plan


    #HDFC Sensex Exchange Traded Fund ----ISIN given




    df.loc[(df.instrument == 'Net Current Assets'),['Coupon','industry/rating','ISIN','qty']] = [0,'additional','additional',1]
    df.loc[(df.instrument == 'Gold - Gold'),['Coupon','industry/rating','ISIN','qty']] = [0,'additional','additional',1]
    
    df.loc[(df.instrument == 'NIFTY NSE PUT OPTION STRIKE PRICE 9600 EXPIR JUN 2020'),['ISIN','industry/rating']] = ['additional','additional']
    df.loc[(df.instrument == 'NIFTY NSE PUT OPTION STRIKE PRICE 9900 EXPIR JUN 2020'),['ISIN','industry/rating']] = ['additional','additional']
    df.loc[(df.instrument == 'HDFC Corporate Bond Fund - Growth Option - Direct Plan'),['ISIN','industry/rating']] = ['additional','additional']
    
    
    for value in df.instrument: 
        #if 'Option' or 'Fund' in str(value):
         #   df.loc[(df.instrument == value),['industry/rating']] = ['additional']
        if 'Repo' in str(value):
            df.loc[(df.instrument == value),['Coupon','industry/rating','ISIN','qty']] = ['0','additional','additional',1]
        
        if 'Ltd.' or 'Limited' or 'Bank' in str(value):
            isin = list(df.loc[(df.instrument == value)]['ISIN'])
            try:
                if isin[-1] is np.nan:
                    df.loc[(df.instrument == value),['Coupon','industry/rating','ISIN','qty']] = ['0','additional','additional',1]
            except IndexError:
                continue
        else:
            continue
            
            

    print(df.isnull())
    df.dropna(inplace = True)
    df.reset_index(drop = True,inplace = True)

    #calculated total
    calculated_total = df['marketval'].sum()
    print("Calculated Total : ", calculated_total)
    
    
    
    
    diff = actual_total-calculated_total 
    if abs(diff) < 1:
        print("Tallied")
    else:
        print("There is a differenc of : ",actual_total - calculated_total )
        not_tallied[fund] = diff
        
        
    if not os.path.exists('HDFC_funds_df/{}'.format(Date)):
        os.makedirs('HDFC_funds_df/{}'.format(Date))
   
    df.to_csv('HDFC_funds_df/{}/{}.csv'.format(Date,fund))
    print('CSV Published')
    count += 1
    
not_tallied

44 HDFCMY
---------
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar

*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255


*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255
*** formula/tFuncVar unknown FuncID:255


{}